In [1]:
!pip install -Uqq transformers sentence-transformers huggingface spacy bert-extractive-summarizer negate
!python -m spacy download en_core_web_md
!pip install -Uqq https://github.com/explosion/spacy-experimental/releases/download/v0.6.1/en_coreference_web_trf-3.4.0a2-py3-none-any.whl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 60.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 84.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 82.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 KB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 95.6 MB/s eta 0:00:00
/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-03-12 11:07:37

In [2]:
main_text = """
The Chrysler Building, the famous art deco New York skyscraper, will be sold for a small fraction of its previous sales price.
The deal, first reported by The Real Deal, was for $150 million, according to a source familiar with the deal.
Mubadala, an Abu Dhabi investment fund, purchased 90% of the building for $800 million in 2008.
Real estate firm Tishman Speyer had owned the other 10%.
The buyer is RFR Holding, a New York real estate company.
Officials with Tishman and RFR did not immediately respond to a request for comments.
It's unclear when the deal will close.
The building sold fairly quickly after being publicly placed on the market only two months ago.
The sale was handled by CBRE Group.
The incentive to sell the building at such a huge loss was due to the soaring rent the owners pay to Cooper Union, a New York college, for the land under the building.
The rent is rising from $7.75 million last year to $32.5 million this year to $41 million in 2028.
Meantime, rents in the building itself are not rising nearly that fast.
While the building is an iconic landmark in the New York skyline, it is competing against newer office towers with large floor-to-ceiling windows and all the modern amenities.
Still the building is among the best known in the city, even to people who have never been to New York.
It is famous for its triangle-shaped, vaulted windows worked into the stylized crown, along with its distinctive eagle gargoyles near the top.
It has been featured prominently in many films, including Men in Black 3, Spider-Man, Armageddon, Two Weeks Notice and Independence Day.
The previous sale took place just before the 2008 financial meltdown led to a plunge in real estate prices.
Still there have been a number of high profile skyscrapers purchased for top dollar in recent years, including the Waldorf Astoria hotel, which Chinese firm Anbang Insurance purchased in 2016 for nearly $2 billion, and the Willis Tower in Chicago, which was formerly known as Sears Tower, once the world's tallest.
Blackstone Group (BX) bought it for $1.3 billion 2015.
The Chrysler Building was the headquarters of the American automaker until 1953, but it was named for and owned by Chrysler chief Walter Chrysler, not the company itself.
Walter Chrysler had set out to build the tallest building in the world, a competition at that time with another Manhattan skyscraper under construction at 40 Wall Street at the south end of Manhattan. He kept secret the plans for the spire that would grace the top of the building, building it inside the structure and out of view of the public until 40 Wall Street was complete.
Once the competitor could rise no higher, the spire of the Chrysler building was raised into view, giving it the title.
"""


In [3]:
import requests
import json
import string
import re
from random import shuffle
import spacy
import torch
import scipy

nlp = spacy.load('en_core_web_md')
# nlp.add_pipe('merge_noun_chunks')
nlp.add_pipe('merge_entities')

nlp_coref = spacy.load("en_coreference_web_trf")

# use replace_listeners for the coref components
nlp_coref.replace_listeners("transformer", "coref", ["model.tok2vec"])
nlp_coref.replace_listeners("transformer", "span_resolver", ["model.tok2vec"])

# we won't copy over the span cleaner
nlp.add_pipe("coref", source=nlp_coref)
nlp.add_pipe("span_resolver", source=nlp_coref)

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/usr/local/lib/python3.9/dist-packages/spacy/language.py:716: UserWarning: [W113] Sourced component 'coref' may not work as expected: source vectors are not identical to current pipeline vectors.
  warnings.warn(Warnings.W113.format(name=source_name))
/usr/local/lib/python3.9/dist-packages/spacy/language.py:716: UserWarning: [W113] Sourced component 'span_resolver' may not work as expected: source vectors are not identical to current pipeline vectors.
  warnings.warn(Warnings.W113.format(name=source_name))


In [4]:
input_text  = """
There is a lot of volcanic activity at divergent plate boundaries in the oceans. For example, many undersea volcanoes are found along the Mid-Atlantic Ridge. This is a divergent plate boundary that runs north-south through the middle of the Atlantic Ocean. As tectonic plates pull away from each other at a divergent plate boundary, they create deep fissures, or cracks, in the crust. Molten rock, called magma, erupts through these cracks onto Earth’s surface. At the surface, the molten rock is called lava. It cools and hardens, forming rock. Divergent plate boundaries also occur in the continental crust. Volcanoes form at these boundaries, but less often than in ocean crust. That’s because continental crust is thicker than oceanic crust. This makes it more difficult for molten rock to push up through the crust. Many volcanoes form along convergent plate boundaries where one tectonic plate is pulled down beneath another at a subduction zone. The leading edge of the plate melts as it is pulled into the mantle, forming magma that erupts as volcanoes. When a line of volcanoes forms along a subduction zone, they make up a volcanic arc. The edges of the Pacific plate are long subduction zones lined with volcanoes. This is why the Pacific rim is called the “Pacific Ring of Fire.”
"""

In [5]:
from spacy.tokens import Doc

# Define lightweight function for resolving references in text
def resolve_references(doc: Doc) -> str:
    """Function for resolving references with the coref ouput
    doc (Doc): The Doc object processed by the coref pipeline
    RETURNS (str): The Doc string with resolved references
    """
    # token.idx : token.text
    token_mention_mapper = {}
    output_string = ""
    clusters = [
        val for key, val in doc.spans.items() if key.startswith("coref_cluster")
    ]

    # Iterate through every found cluster
    for cluster in clusters:
        first_mention = cluster[0]
        # Iterate through every other span in the cluster
        for mention_span in list(cluster)[1:]:
            # Set first_mention as value for the first token in mention_span in the token_mention_mapper
            token_mention_mapper[mention_span[0].idx] = first_mention.text + mention_span[0].whitespace_
            
            for token in mention_span[1:]:
                # Set empty string for all the other tokens in mention_span
                token_mention_mapper[token.idx] = ""

    # Iterate through every token in the Doc
    for token in doc:
        # Check if token exists in token_mention_mapper
        if token.idx in token_mention_mapper:
            output_string += token_mention_mapper[token.idx]
        # Else add original token text
        else:
            output_string += token.text + token.whitespace_

    return output_string

text = [main_text]

txt = [resolve_references(coref_doc) for coref_doc in nlp.pipe(text)]

In [6]:
txt[0]

"\nThe Chrysler Building, the famous art deco New York skyscraper, will be sold for a small fraction of the famous art deco New York skyscraper previous sales price.\nsold was for $150 million, according to a source familiar with sold .\nMubadala, an Abu Dhabi investment fund, purchased 90% of the famous art deco New York skyscraper for $800 million in 2008.\nReal estate firm Tishman Speyer had owned the other 10%.\nThe buyer is RFR Holding, a New York real estate company.\nOfficials with Mubadala, an Abu Dhabi investment fund, and RFR Holding, a New York real estate company did not immediately respond to a request for comments.\nIt's unclear when sold will close.\nthe famous art deco New York skyscraper sold fairly quickly after being publicly placed on the market only two months ago.\nsold was handled by CBRE Group.\nThe incentive to sold the famous art deco New York skyscraper at such a huge loss was due to the soaring rent Mubadala, an Abu Dhabi investment fund, pay to Cooper Union

In [7]:
from summarizer import Summarizer
from transformers import AutoTokenizer, AutoModel, AutoConfig
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
custom_config = AutoConfig.from_pretrained('distilbert-base-uncased')
custom_config.output_hidden_states=True
custom_model = AutoModel.from_pretrained('distilbert-base-uncased', config=custom_config)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
model = Summarizer(custom_model=custom_model, custom_tokenizer=tokenizer)

In [9]:
res = model.calculate_optimal_k(main_text, k_max=10)

/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: F

In [10]:
result = model(input_text, min_length=40, max_length=150)
summarized = ''.join(result)

/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [11]:
summarized_doc = nlp(summarized)
sentence_list = [sent.text for sent in summarized_doc.sents]

In [12]:
from negate import Negator

negator = Negator()

In [13]:
for sentence in sentence_list:
  negated_sentence = negator.negate_sentence(sentence)
  print(negated_sentence)

There isn't a lot of volcanic activity at divergent plate boundaries in the oceans.
Molten rock, called magma, doesn't erupt through these cracks onto Earth’s surface.
Many volcanoes don't form along convergent plate boundaries where one tectonic plate is pulled down beneath another at a subduction zone.


In [24]:
def generate_mask(text):
    doc = nlp(text)
    masked_sentences = []
    for num in range(len(list(doc.noun_chunks))):
        test=''.join(['[MASK]' + token.whitespace_ if token.text in [str(a) for a in list(doc.noun_chunks)][num] and not token.is_stop else token.text + token.whitespace_ for token in doc])
        if '[MASK]' in test:
            masked_sentences.append(test)
    return masked_sentences

In [25]:
text = sentence_list[0]
masked_sentences = generate_mask(text)

In [26]:
masked_sentences

['There is a [MASK] of volcanic activity at divergent plate boundaries in the oceans.',
 'There is a lot of [MASK] [MASK] at divergent plate boundaries in the oceans.',
 'There is a lot of volcanic activity at [MASK] [MASK] [MASK] in the oceans.',
 'There is a lot of volcanic activity at divergent plate boundaries in the [MASK].']

In [16]:
from transformers import AutoModelForMaskedLM
mask_model = AutoModelForMaskedLM.from_pretrained('distilbert-base-uncased', config=custom_config)

In [17]:
def generate_similar_sentences(masked_sentence):
  output_list = []
  inputs = tokenizer(masked_sentence, return_tensors="pt")
  with torch.no_grad():
    logits = mask_model(**inputs).logits

  # retrieve index of [MASK]
  mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]

  predicted_token_id = torch.flip(logits[0, mask_token_index].argsort(axis=-1), [0, 1])[:, :5]
  
  outputs = []
  for token in predicted_token_id:
    output = tokenizer.convert_ids_to_tokens(token)
    outputs.append(output)
    
  for a in zip(*outputs[::-1]):
    sentence = masked_sentence
    for word in a:
      sentence = sentence.replace("[MASK]", word, 1)
    output_list.append(sentence)

  
  return output_list

In [18]:
generated_sentences_list = generate_similar_sentences(masked_sentences[1])

In [19]:
generated_sentences_list

['There is a lot of sediment variability at divergent plate boundaries in the oceans .',
 'There is very lot of oceanic activity at divergent plate boundaries in the oceans .',
 'There is the lot of ocean variation at divergent plate boundaries in the oceans .',
 'There is also lot of seismic occurring at divergent plate boundaries in the oceans .',
 'There is relatively lot of volcanic diversity at divergent plate boundaries in the oceans .']

In [20]:
from sentence_transformers import SentenceTransformer
sent_model = SentenceTransformer('distilbert-base-uncased')

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [21]:
def sort_by_similarity(original_sentence, generated_sentences_list):
    sentence_embeddings = sent_model.encode(generated_sentences_list)

    queries = [original_sentence]
    query_embeddings = sent_model.encode(queries)
    # Find the top sentences of the corpus for each query sentence based on cosine similarity
    number_top_matches = len(generated_sentences_list)

    dissimilar_sentences = []

    for query, query_embedding in zip(queries, query_embeddings):
        distances = scipy.spatial.distance.cdist([query_embedding], sentence_embeddings, "cosine")[0]

        results = zip(range(len(distances)), distances)
        results = sorted(results, key=lambda x: x[1])


        for idx, distance in results[0:3]:
            dissimilar_sentences.append(generated_sentences_list[idx].strip())
          
    
    return dissimilar_sentences

In [22]:
from transformers import pipeline

mask_filler = pipeline(
    "fill-mask", model="ayoolaolafenwa/Masked-Language-Model"
)

preds = mask_filler('Many volcanoes form along [MASK] [MASK] [MASK] where one tectonic plate is pulled down beneath another at a subduction zone')
preds

[[{'score': 0.8973775506019592,
   'token': 1996,
   'token_str': 'the',
   'sequence': '[CLS] many volcanoes form along the [MASK] [MASK] where one tectonic plate is pulled down beneath another at a subduction zone [SEP]'},
  {'score': 0.02743368037045002,
   'token': 2023,
   'token_str': 'this',
   'sequence': '[CLS] many volcanoes form along this [MASK] [MASK] where one tectonic plate is pulled down beneath another at a subduction zone [SEP]'},
  {'score': 0.008662533015012741,
   'token': 1037,
   'token_str': 'a',
   'sequence': '[CLS] many volcanoes form along a [MASK] [MASK] where one tectonic plate is pulled down beneath another at a subduction zone [SEP]'},
  {'score': 0.005439468193799257,
   'token': 2008,
   'token_str': 'that',
   'sequence': '[CLS] many volcanoes form along that [MASK] [MASK] where one tectonic plate is pulled down beneath another at a subduction zone [SEP]'},
  {'score': 0.00499885156750679,
   'token': 2007,
   'token_str': 'with',
   'sequence': '[CLS

In [23]:
sort_by_similarity(text, generated_sentences_list)

['There is relatively lot of volcanic diversity at divergent plate boundaries in the oceans .',
 'There is very lot of oceanic activity at divergent plate boundaries in the oceans .',
 'There is a lot of sediment variability at divergent plate boundaries in the oceans .']